In [3]:
import pandas as pd
import numpy as np
import random
import os
from docx import Document


# implementing random seed for control
sd = 42
np.random.seed(sd)

# school names for consistency 
schools = ['Oakland Middle School',
    'Siegel Middle School',
    'Whitworth-Buchanan Middle School',
    'Christiana Middle School',
    'Smyrna Middle School',
    'Stewarts Creek Middle School',
    'Rockvale Middle School',
    'Rocky Fork Middle School',
    'Blackman Middle School',
    'Thurman Francis Arts Academy',
    'Rock Springs Middle School',
    'LaVergne Middle School'
]

# preparing directories
for school in schools:
    path = f'../YouScienceData/Schedules/{school}'
    schedule_path = f'../YouScienceData/Schedules/{school}/Student_Schedules'
    if not os.path.exists(path):
        os.makedirs(path)
    if not os.path.exists(pathway_path):
        os.makedirs(schedule_path)


In [ ]:
""" 
Creates draft of per-student .docx schedule file, Round 1: Do for one school for proof of concept
FIXED in alpha1.ipynb:
    (1) Read in {school}_all_students.csv file
    (2) Read in Skyward_{school}.xlsx file
    (3) Create a 1st period/Homeroom feature
    (4) AZ by student name (should be default), then AZ by homeroom teacher (ease of school admin)
(5) Begin writing draft schedule template
"""

In [4]:

for school in schools:
    # (1) Read in {school}_all_students.csv file, Now AZ as needed with Homeroom teacher
    school_all_path = f'../YouScienceData/Schedules/{school}/{school}_all_students.csv'
    school_all_df = pd.read_csv(school_all_path)
    school_all_df.drop('Unnamed: 0', axis=1, inplace=True)

    # (5) Need to get POS teacher and room # from planning.xlsx
    # - worksheet name = school name ('Whitworth-Buchanan Middle Schoo')
    if school == 'Whitworth-Buchanan Middle School':
        planning_df = pd.read_excel('planning.xlsx', sheet_name='Whitworth-Buchanan Middle Schoo')
    else:
        planning_df = pd.read_excel('planning.xlsx', sheet_name=school)

    break

## Playground: <i>Student Schedule Print-out</i>

<strong>Objectives:</strong></br>
<ol>
    <li>Pull a student's data for practice.</li>
    <li>Play around with <i>python-docx</i> methods.</li>
    <li>Create reasonable template of test student case.</li>
    <li>Formalize into callable function.</li>
</ol>

In [8]:
planning_df.head()

,POS,MS Room #,MS Teacher,MS Teacher Email,HS Teacher,HS Teacher Email,Zoom Link
0,Agricultural Engineering & Applied Technologiess,603,Defore,DeforeM@rcschools.net,LaLiberte/Sullivan,lalibertec@rcschools.net,https://rcschools.zoom.us/j/85347717350
1,Audio Visual Production,605,McAlexander,mcalexanderc@rcschools.net,Richard Beard,beardr@rcschools.net,https://rcschools.zoom.us/j/86929673851
2,Banking & Finance,604,Dickerson,dickersonr@rcschools.net,Lisa Tate/Wright,tatel@rcschools.net,https://rcschools.zoom.us/j/88557486697
3,BioSTEM,606,Spin,spintzykf@rcschools.net,Missy Bunch,bunchm@rcschools.net,https://rcschools.zoom.us/j/86458950376
4,Business Management,611,Garrison,garrisonc@rcschools.net,Rogers/Jones,rogerssh@rcschools.net,https://rcschools.zoom.us/j/82079830173


In [ ]:
# test case student
test_case = school_all_df.iloc[5]
# Needed featuers: (1) student name, (2) Homeroom Teacher name, (3a-3d) Block Pathway title, 
# (4a-4d) Pathway Instructor, (5a-5d) Pathway room #, (6a-6d) Pathway zoom link
# (1), (2)
student_name, HR_teacher = test_case['name'], test_case['CY Teacher']
# (3a) 
pathway = test_case.B1
POS_info = planning_df.loc[planning_df.POS == pathway]
# 4a, 5a, 6a
POS_instructors = ' & '.join([POS_info['MS Teacher'].values[0], POS_info['HS Teacher'].values[0]])
POS_room_no = POS_info['MS Room #'].values[0]
POS_zoom_link = POS_info['Zoom Link'].values[0]

test_document = Document()
test_document.add_heading(f'Name: {student_name}', level = 1)
test_document.add_paragraph(f'Homeroom: {HR_teacher}')
test_document.add_heading(f'Pathway 1: {pathway}', level = 3)
test_document.add_paragraph(
    f'Sponsor/Instructors: {POS_instructors}\nRoom #: {POS_room_no}\nZoom: {POS_zoom_link}'
)

test_document.save('../test_schedule.docx')

In [30]:
# Test version 2: adding for loop to get each pathway paragraph/heading
test_case2 = school_all_df.iloc[10]
# (1), (2)
student_name, HR_teacher = test_case2['name'], test_case2['CY Teacher']
# Initializing document
test_document2 = Document()
test_document2.add_heading(f'Name: {student_name}', level = 1)
test_document2.add_paragraph(f'Homeroom: {HR_teacher}')

for block in ['B1','B2','B3','B4']: 
    # get pathway's info
    # 3a
    pathway = test_case2[block]
    POS_info = planning_df.loc[planning_df.POS == pathway]
    # 4a, 5a, 6a
    POS_instructors = ' & '.join([POS_info['MS Teacher'].values[0], POS_info['HS Teacher'].values[0]])
    POS_room_no = POS_info['MS Room #'].values[0]
    POS_zoom_link = POS_info['Zoom Link'].values[0]

    # structure document
    test_document2.add_heading(f'Pathway {block[1]}: {pathway}', level=3)
    test_document2.add_paragraph(
        f'Sponsor/Instructors: {POS_instructors}\nRoom #: {POS_room_no}\nZoom: {POS_zoom_link}'
    )

test_document2.save('../test_schedule2.pdf')

### Editing, Checking POS/YS Clusters were translated correctly

In [ ]:
# Checking pathways are established correctly
planning_pos_list = list(planning_df.POS)
df_pos_list = list(school_all_df.B1.unique())
for pos in df_pos_list:
    if pos not in planning_pos_list:
        check = str(pos[:5])
        print(pos)
        for option in planning_pos_list:
            if check in option:
                if option in df_pos_list:
                    swap = 'Already in use.'
                else:
                    swap = 'Can switch!'
                print(school, option, swap)

In [ ]:
# # borrowed from alpha1.ipynb
# def last_name(name):
#     return name.split(',')[0].split()[-1]

# for school in schools:
#     # (1) Read in {school}_all_students.csv file
#     school_all_path = f'../YouScienceData/Schedules/{school}/{school}_all_students.csv'
#     school_all_df = pd.read_csv(school_all_path)
#     school_all_df.drop('Unnamed: 0', axis=1, inplace=True)

#     # (2) Read in Skyward_{school}.xlsx file
#     school_skyward_path = f'../YouScienceData/Skyward/Skyward_{school}.xlsx'
#     school_skyward_df = pd.read_excel(school_skyward_path)

#     # (3) Create a 1st period/Homeroom feature
#     for email in list(school_all_df.email):
#         if 'Open' in email:
#             continue
        
#         X = school_skyward_df.loc[school_skyward_df['Student\'s School Email'] == email]
#         teacher_last, teacher_first = X['CY Teacher Last Name'].values[0], X['CY Teacher First Name'].values[0]
#         school_all_df.loc[school_all_df.email == email, 'CY Teacher'] = teacher_last + ', ' + teacher_first[0]
        
#     # (4) AZ by student name (should be default), then AZ by homeroom teacher (ease of school admin)
#     school_all_df['Last'] = school_all_df['name'].apply(last_name)
#     school_all_df.sort_values(['CY Teacher', 'Last'], inplace=True)
#     school_all_df.index = np.arange(1, len(school_all_df) + 1)
#     break